In [1]:
import polars as pl
from scipy.stats import binom
import pathlib

## Load the raw response data

Each Excel file uses a slightly different layout and variable names, so the following functions standardize a few key columns:

- Grant ID
- Provider name
- Response date
- Q1 response (as a number)

In [2]:
def fix_cols_1(df):
    return (
        df
        .with_columns(
            pl.col("EndDate").cast(int).alias("date_int")
        )
        .rename({
            "Custom Variable 1": "provider_name",
            "Custom Variable 2": "grant_id",
        })
    )

def fix_cols_2(df):
    return (
        df
        .drop("Q1")
        .rename({
            "GRANTEEID": "grant_id",
            "LtCallDt": "date_int",
            "Q1 Num": "Q1",
        })
        .with_columns(
            pl.lit(None).cast(str).alias("provider_name")
        )
    )

def fix_cols_3(df):
    return (
        df
        .rename({
            "LEGAL_NAME": "provider_name",
            "GRANT_ID": "grant_id",
            "Timestamp": "date_int",
        })
    )

In [3]:
def load_responses(path):
    df = pl.read_excel(
        path,
        read_csv_options=dict(infer_schema_length=0),
        xlsx2csv_options=dict(dateformat="%Y%m%d"),
    )
    if "Q8.4-c_6_TEXT" in df.columns:
        fixer = fix_cols_1
    elif "GRANTEEID" in df.columns:
        fixer = fix_cols_2
    else:
        fixer = fix_cols_3
        
    return (
        df
        .pipe(fixer)
        .with_columns(
            pl.col("Q1").cast(int),
            pl.col("date_int").cast(str).str.strptime(pl.Date, "%Y%m%d").alias("date")
        )
        .with_columns(
            (pl.col("date").dt.year() + (pl.col("date").dt.month() >= 10)).alias("fiscal_year")
        )
        .select(
            "grant_id",
            "provider_name",
            "date",
            "fiscal_year",
            "Q1"
        )
    )

In [4]:
paths = sorted(pathlib.Path("../data/raw/").glob("*.xlsx"))
paths

[PosixPath('../data/raw/FINAL Data - FY 2016-2017_Excluding PII.xlsx'),
 PosixPath('../data/raw/FINAL Data - FY 2018-2020_Excluding PII.xlsx'),
 PosixPath('../data/raw/SSVF Annual National Report Data Oct 21 - Sep 22_Excluding PII.xlsx')]

In [5]:
responses = pl.concat(map(load_responses, paths))
responses.glimpse()

Rows: 39544
Columns: 5
$ grant_id       <str> 13-GA-102, 13-NV-118, 14-CA-169, 14-MA-209, 14-AR-156, 14-ZZ-153, 14-AL-155, 14-NC-235, C2015-DC-500B, 14-WV-303
$ provider_name  <str> Decatur Cooperative Ministry  Inc., Salvation Army  a California Corporation, Families in Transition of Santa Cruz County  Inc., Veterans Northeast Outreach Center  Inc., Seven Hills Homeless Center, United Way of Central Alabama, Family Endeavors  Inc., Family Endeavors  Inc., Friendship Place, The Greater Wheeling Coalition for the Homeless  Inc.
$ date          <date> 2016-05-01, 2016-05-01, 2016-05-01, 2016-05-01, 2016-05-01, 2016-05-01, 2016-05-01, 2016-05-01, 2016-05-01, 2016-05-01
$ fiscal_year    <i32> 2016, 2016, 2016, 2016, 2016, 2016, 2016, 2016, 2016, 2016
$ Q1             <i64> 1, 4, 4, 4, 2, 5, 3, 4, 4, 5



In [6]:
assert responses["Q1"].is_null().mean() == 0

Total responses:

In [7]:
len(responses)

39544

By fiscal year:

In [8]:
(
    responses
    ["fiscal_year"]
    .value_counts()
)

fiscal_year,counts
i32,u32
2016,2766
2017,5696
2018,10137
2019,8859
2020,7464
2022,4622


By quearter:

In [9]:
(
    responses
    .sort("date")
    .groupby_dynamic(
        pl.col("date").alias("quarter"),
        every="1q"
    )
    .agg(pl.count())
)

quarter,count
date,u32
2016-01-01,150
2016-04-01,1174
2016-07-01,1442
2016-10-01,1165
2017-01-01,1401
2017-04-01,1349
2017-07-01,1781
2017-10-01,1898
2018-01-01,2799


## Check data records vs. aggregates in PDF for FY2020

In [10]:
responses_fy20 = (
    responses
    .filter(pl.col("fiscal_year") == 2020)
)

len(responses_fy20)

7464

In [11]:
pdf_counts_fy20 = pl.read_csv("../data/from-pdfs/provider-satisfaction-fy20.csv")
pdf_counts_fy20.head()

grant_id,provider_name,responses_total,responses_top_2,performance_rating
str,str,i64,i64,f64
"""18-ZZ-034""","""Volunteers of …",43,22,0.0
"""14-MD-214""","""Project PLASE …",35,18,0.1
"""14-NY-259""","""Volunteers of …",13,4,0.1
"""14-ZZ-153""","""United Way of …",23,10,0.1
"""20-CA-017""","""The Salvation …",27,13,0.1


Note: "performance rating" is a calculation described on p. 15 the FY20 PDF report; it represents the likelihood that an "average" provider would receive fewer top-2 (i.e., "Excellent" or "Above Average") responses to Q1 than this particular provider, given the number of respondents for the provider. In the cell below, we confirm the calculation, which we implement via `scipy`'s `binom.cdf(numerator, denominator, propensity)` method.

In [12]:
# From examples in FY20 report, p. 15
assert round(binom.cdf(8, 10, 0.7), 3) == 0.851
assert round(binom.cdf(150, 200, 0.7), 3) == 0.949

By the Data Liberation Project's reading of the methodology, the "average provider"'s propensity for top-2 responses should be equal to the overall proportion of top-2 responses, calculated in the following cell. (This number matches up with the PDF, which notes, "The systemwide percentage of responses to Q1 that were in the Top 2 [...] was 74.4%." But as demonstrated further below, it seems a different average was used for the performance ratings in the report's Appendix A.

In [13]:
POPULATION_MEAN_FY20 = responses_fy20["Q1"].is_in([ 4, 5 ]).mean()
round(POPULATION_MEAN_FY20, 3)

0.744

In [14]:
MEAN_USED = 0.761905
metrics_fy20 = (
    responses_fy20
    .groupby("grant_id")
    .agg(
        pl.count().alias("responses_total"),
        pl.col("Q1").is_in([ 4, 5 ]).sum().alias("responses_top_2"),
    )
    .with_columns(
        (
            pl.struct(["responses_top_2", "responses_total"])
            .apply(lambda row: round(100 * binom.cdf(row["responses_top_2"], row["responses_total"], MEAN_USED), 1))
            .alias("performance_rating")
        )
    )
    .sort("responses_total", descending=True)
)

metrics_fy20.head()

grant_id,responses_total,responses_top_2,performance_rating
str,u32,u32,f64
"""20-ZZ-026""",249,225,100.0
"""20-ZZ-113""",175,147,99.6
"""19-FL-025""",169,121,9.7
"""19-CA-009""",153,101,0.3
"""18-TX-074""",145,110,49.5


In [15]:
metrics_fy20_comparison = (
    metrics_fy20
    .join(
        pdf_counts_fy20.select(pl.exclude("provider_name")),
        how = "outer",
        on = "grant_id",
        suffix = "_pdf",
    )
)

metrics_fy20_comparison.head()

grant_id,responses_total,responses_top_2,performance_rating,responses_total_pdf,responses_top_2_pdf,performance_rating_pdf
str,u32,u32,f64,i64,i64,f64
"""18-ZZ-034""",43,22,0.0,43,22,0.0
"""14-MD-214""",35,18,0.1,35,18,0.1
"""14-NY-259""",13,4,0.1,13,4,0.1
"""14-ZZ-153""",23,10,0.1,23,10,0.1
"""20-CA-017""",27,13,0.1,27,13,0.1


In [16]:
mismatched = (
    metrics_fy20_comparison
    .filter(
        (pl.col("performance_rating") != pl.col("performance_rating_pdf")) |
        (pl.col("responses_total") != pl.col("responses_total_pdf")) |
        (pl.col("responses_top_2") != pl.col("responses_top_2_pdf"))
    )
)

assert len(mismatched) == 0

## Generalizing to produce provider aggregates for other years

In [17]:
# See addendum below

latest_names = (
    responses
    .filter(~pl.col("provider_name").is_null())    
    .sort("date")
    .groupby("grant_id")
    .agg(pl.col("provider_name").last().alias("provider_name_latest"))
)

latest_names.head()

grant_id,provider_name_latest
str,str
"""C15-NY-600B""","""HELP Social Se…"
"""14-MD-214""","""Project PLASE …"
"""16-NY-057""","""Veterans Outre…"
"""18-OH-263""","""Talbert House …"
"""14-CA-163""","""Carrillo Couns…"


In [18]:
results_by_fiscal_year = (
    responses
    .sort("date")
    .groupby(
        "fiscal_year",
        "grant_id",
    )
    .agg(
        pl.col("provider_name").last(),
        pl.count().alias("responses_total"),
        *(pl.col("Q1").eq(x).alias(f"q1_{x}").sum() for x in range(1, 6)),
        pl.col("Q1").is_in([ 4, 5 ]).sum().alias("top_2_count"),
    )
    .with_columns(
        (100 * pl.col("top_2_count") / pl.col("responses_total")).round(1).alias("top_2_pct"),
    )
    .sort("fiscal_year", "grant_id", descending=(True, False))
    .join(latest_names, on="grant_id")
    .select(
        "fiscal_year",
        "grant_id",
        "provider_name",
        "provider_name_latest",
        "responses_total",
        *(f"q1_{x}" for x in range(1, 6)),
        "top_2_count",
        "top_2_pct",
    )
)

results_by_fiscal_year.head()

fiscal_year,grant_id,provider_name,provider_name_latest,responses_total,q1_1,q1_2,q1_3,q1_4,q1_5,top_2_count,top_2_pct
i32,str,str,str,u32,u32,u32,u32,u32,u32,u32,f64
2022,"""12-AK-001""",null,"""Catholic Socia…",2,0,0,0,0,2,2,100.0
2022,"""12-AL-002""",null,"""Housing First …",2,0,0,0,1,1,2,100.0
2022,"""12-CA-013""",null,"""Goodwill of Si…",6,1,0,0,0,5,5,83.3
2022,"""12-CA-016""",null,"""Shelter Inc of…",2,0,0,0,1,1,2,100.0
2022,"""12-CA-018""",null,"""Volunteers of …",19,1,1,1,7,9,16,84.2


In [19]:
(results_by_fiscal_year)

fiscal_year,grant_id,provider_name,provider_name_latest,responses_total,q1_1,q1_2,q1_3,q1_4,q1_5,top_2_count,top_2_pct
i32,str,str,str,u32,u32,u32,u32,u32,u32,u32,f64
2022,"""12-AK-001""",null,"""Catholic Socia…",2,0,0,0,0,2,2,100.0
2022,"""12-AL-002""",null,"""Housing First …",2,0,0,0,1,1,2,100.0
2022,"""12-CA-013""",null,"""Goodwill of Si…",6,1,0,0,0,5,5,83.3
2022,"""12-CA-016""",null,"""Shelter Inc of…",2,0,0,0,1,1,2,100.0
2022,"""12-CA-018""",null,"""Volunteers of …",19,1,1,1,7,9,16,84.2
2022,"""12-CT-021""",null,"""Community Rene…",6,0,0,0,2,4,6,100.0
2022,"""12-FL-024""",null,"""Advocate Progr…",29,8,3,1,3,14,17,58.6
2022,"""12-FL-028""",null,"""Jewish Family …",6,0,1,0,1,4,5,83.3
2022,"""12-GA-029""",null,"""Central Savann…",23,0,1,7,4,11,15,65.2


In [20]:
results_by_fiscal_year.write_csv(
    "../data/output/provider-satisfaction-by-fiscal-year.csv"
)

## Addendum note on provider names

Names generally stay the same, with stylistic changes, but not always:

In [21]:
(
    responses
    .filter(~pl.col("provider_name").is_null())
    .with_columns(
        (
            pl.col("provider_name")
            .str.to_uppercase()
            .str.replace_all(r"[,.]", "")
            .str.replace_all(r"\s+", " ")
            .str.replace_all(r" INC\s*", "")
            .str.strip()
            .alias("name_normalized")
        )
    )
    .groupby("grant_id")
    .agg(
        pl.col("name_normalized").n_unique().alias("total_names"),
        pl.col("name_normalized").unique().alias("names")
    )
    .filter(pl.col("total_names") > 1)
)

grant_id,total_names,names
str,u32,list[str]
"""13-IN-106""",2,"[""COMMUNITY ACTION OF NORTHEAST INDIANA(DBA BRIGHTPOINT)"", ""COMMUNITY ACTION OF NORTHEAST INDIANA""]"
"""14-NY-251""",2,"[""VETERANS AND COMMUNITY HOUSING COALITION"", ""SARATOGA COUNTY RURAL PRESERVATION COMPANY""]"
"""14-MI-218""",2,"[""BLUEWATER CENTER FOR INDEPENDENT LIVING"", ""BLUE WATER CENTER FOR INDEPENDENT LIVING""]"
"""12-FL-025""",2,"[""CARRFOUR SUPPORTIVE HOUSING"", ""PURPOSE BUILT FAMILIES FOUNDATION""]"
"""12-NY-063""",2,"[""WESTCHESTER COMMUNITY OPPORTUNITY PROGRAM(WESTCOP)"", ""WESTCHESTER COMMUNITY OPPORTUNITY PROGRAM""]"
"""12-OH-064""",2,"[""FRONTLINE"", ""MENTAL HEALTH SERVICES FOR HOMELESS PERSONSDBA FRONTLINE SERVICE""]"
"""13-NC-114""",2,"[""ASHEVILLE BUNCOMBE COMMUNITY CHRISTIAN MINISTRY"", ""ASHEVILLE-BUNCOMBE COMMUNITY CHRISTIAN MINISTRY""]"
"""C15-NV-500B""",2,"[""UNITED STATES VETERANS INITIATIVE"", ""USVETS LAS VEGAS""]"
"""12-ZZ-020""",2,"[""ROCKY MOUNTAIN HUMAN SERVICES (DBA DENVER OPTIONS"", ""ROCKY MOUNTAIN HUMAN SERVICES (DBA DENVER OPTIONS)""]"


---

---

---
